## Library and COBRA model imports 

In [1]:
import thermodynamic_model_05
import cobra

bigg_model_path = '..\COBRA function scripts\e_coli_core metabolism from BiGG.json'
model = cobra.io.load_json_model(bigg_model_path)

## Creating a sample thermodynamic dataset  

A sample dataset is created that will be passed through the following example

In [2]:
import pandas
import re

# acquire Gibbs reactions data from the TMFA supplementary excel file        
reactions_data = pandas.read_excel('Supplementary file.xls', sheet_name = 'Reactions')
reactions_data.columns = reactions_data.iloc[0]
reactions_data.drop(reactions_data.index[0], axis = 0, inplace = True)
reactions_data.head()
reactions_dict = {}
for index, row in reactions_data.iterrows():
    reaction_abbreviation = reactions_data.at[index, 'iJR904 Abbreviation']
    reaction_name = reactions_data.at[index, 'Reaction name']
    reaction_gibbs = reactions_data.at[index, 'Estimated Gibbs free energy change of reaction (kcal/mol)']

    reactions_dict[reaction_abbreviation] = {'name': reaction_name, 'gibbs': reaction_gibbs}

# acquire Gibbs compounds data from the TMFA supplementary excel file        
compounds_data = pandas.read_excel('Supplementary file.xls', sheet_name = 'Compounds')
compounds_dict = {}
for index, row in compounds_data.iterrows():
    compound_abbreviation = compounds_data.at[index, 'iJR904 Abbreviation']
    compound_name = compounds_data.at[index, 'Compound Name']
    compound_charge = compounds_data.at[index, 'Charge at pH 7*']
    try:
        compound_gibbs = float(compounds_data.at[index, 'Estimated Gibbs free energy of formation (kcal/mol)']) / calorie
    except:
        compound_gibbs = 0

    compounds_dict[compound_abbreviation] = {'name': compound_name, 'gibbs': compound_gibbs, 'charge': compound_charge}

# Base FBA functionality

The Base FBA class introduces the essential functions that are inherited into the following functions.  

In [3]:
model = cobra.io.load_json_model(bigg_model_path)
announcement = 'BaseFBApkg'
print(announcement, '\n', '='*len(announcement))


#--------------------------- __init__ ---------------------------------
print('\n+ __init__')

base = thermodynamic_model_05.BaseFBAPkg(model = model, name = 'test_model')    
    
print('model used: ', base.model)
print('instance name: ', base.name)
print('variable types: ', base.variable_types)
print('constraint types: ', base.constraint_types)


#-------------------------- validate_parameters -----------------------
print('\n+ validate_parameters')

params = {'a':2, 'b':4, 'c':3}
required = ['a', 'b', 'c']
defaults = {'a':1, 'b':1, 'c':1, 'd':1}

print('parameters before: ', base.parameters)
base.validate_parameters(params, required, defaults)
print('parameters after: ', base.parameters)


#-------------------------- build_variable -----------------------------
print('\n+ build_variable')
print('variables before: ', base.variables)

base.build_variable(kind = 'concentration', lower_bound = 10, upper_bound = 133, vartype = 'continuous', obj = model.reactions.get_by_id('PFK'))

print('variables after: ', base.variables)


#--------------------------- build_constraint --------------------------
print('\n+ build_constraint')
print('constraints before: ', base.constraints)

base.build_constraint(constraint_expression = 0, kind = 'concentration', lower_bound = 10, upper_bound = 133, obj = model.reactions.get_by_id('PFK'), coef = {})

print('constraints after: ', base.constraints)


#---------------------------- all_variables -----------------------------
print('\n+ all_variables')
print('model variables: ', base.all_variables())


#--------------------------- all_constraints ----------------------------
print('\n+ all_constraints')
print('model constraints: ', base.all_constraints())


#--------------------------- revert_to_original --------------------------
print('\n+ revert_to_original')

model2 = cobra.io.load_json_model(bigg_model_path)

variables = []
constraints = []
for var in model.variables:
    variables.append(str(var))
for constraint in model.constraints:
    constraints.append(str(constraint))
    
variables2 = []
constraints2 = []
for var in model2.variables:
    variables2.append(str(var))
for constraint in model2.constraints:
    constraints2.append(str(constraint))
    
difference_variables = set(variables) - set(variables2)
print('added variables before: ', difference_variables)
difference_constraints = set(constraints) - set(constraints2)
print('added constraints before: ', difference_constraints)

model = base.revert_to_original(cobra_model_path = bigg_model_path)

variables = []
constraints = []
for var in model.variables:
    variables.append(str(var))
for constraint in model.constraints:
    constraints.append(str(constraint))
    
variables2 = []
constraints2 = []
for var in model2.variables:
    variables2.append(str(var))
for constraint in model2.constraints:
    constraints2.append(str(constraint))
    
difference_variables = set(variables) - set(variables2)
print('added variables after: ', difference_variables)
difference_constraints = set(constraints) - set(constraints2)
print('added constraints after: ', difference_constraints)

BaseFBApkg 

+ __init__
model used:  e_coli_core
instance name:  test_model
variable types:  {'concentration': 'float', 'lnconc': 'float'}
constraint types:  {'concentration': 'float'}

+ validate_parameters
parameters before:  {}
parameters after:  {'a': 2, 'b': 4, 'c': 3, 'd': 1}

+ build_variable
variables before:  {'concentration': {}, 'lnconc': {}}
variables after:  {'concentration': {'PFK': 10 <= PFK_concentration <= 133}, 'lnconc': {}}

+ build_constraint
constraints before:  {'concentration': {}}
{}
constraints after:  {'concentration': {'PFK': <optlang.cplex_interface.Constraint object at 0x0000024D55D46B08>}}

+ all_variables
model variables:  {'concentration': {'PFK': 10 <= PFK_concentration <= 133}, 'lnconc': {}}

+ all_constraints
model constraints:  {'concentration': {'PFK': <optlang.cplex_interface.Constraint object at 0x0000024D55D46B08>}}

+ revert_to_original
added variables before:  {'10 <= PFK_concentration <= 133'}
added constraints before:  {'PFK_concentration: 10

# RevBin functionality

A reaction is evaluated as a variable for the forward and backward fluxes. 

In [4]:
model = cobra.io.load_json_model(bigg_model_path)
announcement = 'Revbin'
print('\n', announcement, '\n', '='*len(announcement))


#--------------------------- __init__ ---------------------------------
print('\n+ __init__')

revbin = thermodynamic_model_05.RevBinPkg(model, obj = model.reactions.get_by_id('PFK'))

print('model used: ', revbin.model)
print('instance name: ', revbin.name)
print('variable types: ', revbin.variable_types)
print('constraint types: ', revbin.constraint_types)


#------------------------- build_constraint ---------------------------
print('\n+ build_constraint')
print('constraints before: ', revbin.constraints)

revbin.build_constraint(obj = model.reactions.get_by_id('PFK'))

print('constraints after: ', revbin.constraints)


#------------------------- build_package ------------------------------
print('\n+ build_package')
constraints = []
for constraint in revbin.constraints:
    for datam in revbin.constraints[constraint]:
        constraints.append(str(datam))
print('constraints before: ', len(constraints))

revbin.build_package(filter = constraints)

constraints = []
for constraint in revbin.constraints:
    for datam in revbin.constraints[constraint]:
        constraints.append(str(datam))
print('constraints after: ', len(constraints))


#------------------------- revert_to_original --------------------------
model = revbin.revert_to_original(cobra_model_path = bigg_model_path)


 Revbin 

+ __init__
model used:  e_coli_core
instance name:  reversible binary
variable types:  {'revbin': 'reaction', 'forv': 'reaction', 'revv': 'reaction'}
constraint types:  {'revbinF': 'reaction', 'revbinR': 'reaction'}

+ build_constraint
constraints before:  {'revbinF': {}, 'revbinR': {}}
{0 <= PFK_revbin <= 1: -1000, 0 <= PFK <= 1000.0: 1}
{0 <= PFK_revbin <= 1: 1000, 0 <= PFK_reverse_d24a6 <= -0.0: 1}
constraints after:  {'revbinF': {'PFK': <optlang.cplex_interface.Constraint object at 0x0000024D56F3C648>}, 'revbinR': {'PFK': <optlang.cplex_interface.Constraint object at 0x0000024D56F6D748>}}

+ build_package
constraints before:  2
{0 <= PFL_revbin <= 1: -1000, 0 <= PFL <= 1000.0: 1}
{0 <= PFL_revbin <= 1: 1000, 0 <= PFL_reverse_af9ec <= -0.0: 1}
{0 <= PGI_revbin <= 1: -1000, 0 <= PGI <= 1000.0: 1}
{0 <= PGI_revbin <= 1: 1000, 0 <= PGI_reverse_27efc <= 1000.0: 1}
{0 <= PGK_revbin <= 1: -1000, 0 <= PGK <= 1000.0: 1}
{0 <= PGK_revbin <= 1: 1000, 0 <= PGK_reverse_02696 <= 1000.

# Simple Thermo functionality

The thermodynamic perspectives of each object metabolite are determined. The class parameterizes each metabolite as a variable, and thereby presumes nothing of the metabolite thermodynamics.  

In [5]:
import thermodynamic_model_02
import re

model = cobra.io.load_json_model(bigg_model_path)
announcement = 'Simple Thermo'
print('\n', announcement, '\n', '='*len(announcement))


#--------------------------- __init__ ---------------------------------
print('\n+ __init__')
simple_thermo = thermodynamic_model_05.SimpleThermoPkg(model, thermo_compounds = compounds_dict)
print('model used: ', simple_thermo.model)
print('instance name: ', simple_thermo.name)
print('variable types: ', simple_thermo.variable_types)
print('constraint types: ', simple_thermo.constraint_types)


#-------------------------- build_constraint ---------------------------
print('\n+ build_constraint')
print('constraints before: ', simple_thermo.constraints)
simple_thermo.build_constraint(obj = model.reactions.get_by_id('PFK'))
print('constraints after: ', simple_thermo.constraints)


#--------------------------- build_package -----------------------------
print('\n+ build_package')
before_constraints = 0
for kind, value in simple_thermo.constraints.items():
    for constraint in value:
        print(value[constraint])
        before_constraints += 1
print('constraints before: ', before_constraints)
    
simple_thermo.build_package(view_errors = False)

after_constraints = 0
for kind, value in simple_thermo.constraints.items():
    for constraint in value:
        print(value[constraint])
        after_constraints += 1
print('constraints after: ', after_constraints)    

#-------------------------- revert_to_original -------------------------
model = simple_thermo.revert_to_original(cobra_model_path = bigg_model_path)


 Simple Thermo 

+ __init__
model used:  e_coli_core
instance name:  simple thermo
variable types:  {'potential': 'metabolite', 'revbin': 'reaction', 'forv': {}, 'revv': {}}
constraint types:  {'simple_thermo': 'reaction', 'revbinF': {}, 'revbinR': {}}

+ build_constraint
constraints before:  {'simple_thermo': {}, 'revbinF': {}, 'revbinR': {}}
{0 <= PFK_revbin <= 1: -1000, 0 <= PFK <= 1000.0: 1}
{0 <= PFK_revbin <= 1: 1000, 0 <= PFK_reverse_d24a6 <= -0.0: 1}
{0 <= PFK_revbin <= 1: 1000, 0 <= adp_c_potential <= 1000: 1.0, 0 <= atp_c_potential <= 1000: -1.0, 0 <= f6p_c_potential <= 1000: -1.0, 0 <= fdp_c_potential <= 1000: 1.0, 0 <= h_c_potential <= 1000: 1.0}
constraints after:  {'simple_thermo': {'PFK': <optlang.cplex_interface.Constraint object at 0x0000024D57656188>}, 'revbinF': {'PFK': <optlang.cplex_interface.Constraint object at 0x0000024D575DBBC8>}, 'revbinR': {'PFK': <optlang.cplex_interface.Constraint object at 0x0000024D5786B808>}}

+ build_package
PFK_simple_thermo: 0 <= 100

GLUSy_simple_thermo: 0 <= 1000.0*GLUSy_revbin - 1.0*akg_c_potential - 1.0*gln__L_c_potential + 2.0*glu__L_c_potential - 1.0*h_c_potential + 1.0*nadp_c_potential - 1.0*nadph_c_potential <= 1000
GLUt2r_simple_thermo: 0 <= 1000.0*GLUt2r_revbin + 1.0*glu__L_c_potential - 1.0*glu__L_e_potential + 1.0*h_c_potential - 1.0*h_e_potential <= 1000
GND_simple_thermo: 0 <= -1.0*6pgc_c_potential + 1000.0*GND_revbin + 1.0*co2_c_potential - 1.0*nadp_c_potential + 1.0*nadph_c_potential + 1.0*ru5p__D_c_potential <= 1000
H2Ot_simple_thermo: 0 <= 1000.0*H2Ot_revbin + 1.0*h2o_c_potential - 1.0*h2o_e_potential <= 1000
ICDHyr_simple_thermo: 0 <= 1000.0*ICDHyr_revbin + 1.0*akg_c_potential + 1.0*co2_c_potential - 1.0*icit_c_potential - 1.0*nadp_c_potential + 1.0*nadph_c_potential <= 1000
ICL_simple_thermo: 0 <= 1000.0*ICL_revbin + 1.0*glx_c_potential - 1.0*icit_c_potential + 1.0*succ_c_potential <= 1000
LDH_D_simple_thermo: 0 <= 1000.0*LDH_D_revbin + 1.0*h_c_potential - 1.0*lac__D_c_potential - 1.0*nad_c_poten

# Full thermo functionality

The thermodynamic perspectives of each object metabolite are determined. The class calculates the total energy for each metabolite, which is the sum of each Gibbs potential, concentration potential, and electrochemical potential.

In [6]:
import thermodynamic_model_05
import cobra

bigg_model_path = '..\COBRA function scripts\e_coli_core metabolism from BiGG.json'
model = cobra.io.load_json_model(bigg_model_path)

announcement = 'Full Thermo'
print('\n', announcement, '\n', '='*len(announcement))



#--------------------------- __init__ ---------------------------------
print('\n+ __init__')

full_thermo = thermodynamic_model_05.FullThermoPkg(model, thermo_compounds = compounds_dict, thermodynamics_data_type = 'dictionary')

print('model used: ', full_thermo.model)
print('instance name: ', full_thermo.name)
print('variable types: ', full_thermo.variable_types)
print('constraint types: ', full_thermo.constraint_types)


#------------------------- build_constraint --------------------------
print('\n+ build_constraint')
print('constraints before: ', full_thermo.constraints)

full_thermo.build_constraint(obj = model.reactions.get_by_id('PFK'))

print('constraints after: ', full_thermo.constraints)
#full_thermo.model.constraints


#------------------------- build_package -----------------------------
print('\n+ build_package')
print('constraints before: ', full_thermo.constraints)

full_thermo.build_package(view_errors = False)

constraints = []
for constraint in full_thermo.constraints:
    for datam in full_thermo.constraints[constraint]:
        constraints.append(str(datam))
print('constraints after: ', len(constraints))


#------------------------- write_lp_file -----------------------------
from datetime import date
import os 

print('\n+ write_lp_file')
export_filename = 'Full thermo test'
import_iteration = 0
filename = '{}_{}_{}.lp'.format(date.today(), export_filename, import_iteration)
print('LP file existing before: ', os.path.exists(filename))

full_thermo.write_lp_file(model = model, export_filename = export_filename)

print('LP file existing after: ', os.path.exists(filename)) 
#os.remove(filename)

#------------------------ revert_to_original --------------------------
model = full_thermo.revert_to_original(cobra_model_path = bigg_model_path)


 Full Thermo 

+ __init__
model used:  e_coli_core
instance name:  full_thermo
variable types:  {'revbin': {}, 'forv': {}, 'revv': {}, 'ln_concentration': {}, 'potential': {}}
constraint types:  {'concentration_potential': 'metabolite', 'electrochemical_potential': 'metabolite', 'full_thermo': 'metabolite', 'revbinF': {}, 'revbinR': {}, 'simple_thermo': {}}

+ build_constraint
constraints before:  {'concentration_potential': {}, 'electrochemical_potential': {}, 'full_thermo': {}, 'revbinF': {}, 'revbinR': {}, 'simple_thermo': {}}
{0 <= PFK_revbin <= 1: -1000, 0 <= PFK <= 1000.0: 1}
{0 <= PFK_revbin <= 1: 1000, 0 <= PFK_reverse_d24a6 <= -0.0: 1}
{0.001 <= adp_c_ln_concentration <= 20: 1.0, 0.001 <= atp_c_ln_concentration <= 20: -1.0, 0.001 <= f6p_c_ln_concentration <= 20: -1.0, 0.001 <= fdp_c_ln_concentration <= 20: 1.0, 0.0053 <= h_c_ln_concentration <= 0.0053: -0.020202707317519466, 0 <= PFK_revbin <= 1: 1000}
constraints after:  {'concentration_potential': {}, 'electrochemical_poten

{0.001 <= succ_e_ln_concentration <= 20: -1.0, 0 <= EX_succ_e_revbin <= 1: 1000}
{0.001 <= dhap_c_ln_concentration <= 20: 1.0, 0.001 <= fdp_c_ln_concentration <= 20: -1.0, 0.001 <= g3p_c_ln_concentration <= 20: 1.0, 0 <= FBA_revbin <= 1: 1000}
{0.001 <= f6p_c_ln_concentration <= 20: 1.0, 0.001 <= fdp_c_ln_concentration <= 20: -1.0, 0.001 <= h2o_c_ln_concentration <= 20: -1.0, 0.001 <= pi_c_ln_concentration <= 20: 1.0, 0 <= FBP_revbin <= 1: 1000}
{0.001 <= for_c_ln_concentration <= 20: 1.0, 0.001 <= for_e_ln_concentration <= 20: -1.0, 0.0053 <= h_c_ln_concentration <= 0.0053: -0.020202707317519466, 0.0053 <= h_e_ln_concentration <= 0.0053: 0.020202707317519466, 0 <= FORt2_revbin <= 1: 1000}
{0.001 <= for_c_ln_concentration <= 20: 1.0, 0.001 <= for_e_ln_concentration <= 20: -1.0, 0 <= FORt_revbin <= 1: 1000}
{0.001 <= fum_c_ln_concentration <= 20: -1.0, 0.001 <= q8_c_ln_concentration <= 20: 1.0, 0.001 <= q8h2_c_ln_concentration <= 20: -1.0, 0.001 <= succ_c_ln_concentration <= 20: 1.0, 0 

# Brainstorming

In [7]:
simple_thermo = thermodynamic_model_04.SimpleThermoPkg(model, object = model.reactions.get_by_id('PFK'), thermo_reactions = reactions_dict, thermo_compounds = compounds_dict)

for item, value in simple_thermo.thermo_compounds.items(): 
    print(item, ': ', value)

NameError: name 'thermodynamic_model_04' is not defined

In [ ]:
reaction = model.reactions.get_by_id('PFK').metabolites
for metabolite in reaction:
    print(metabolite)
    print(reaction[metabolite.id])